In [61]:
from pymilvus import connections, Collection, utility
from pymilvus import utility
import google.generativeai as genai
from local_settings import API_KEY
from pymilvus import MilvusClient
from helper_func import create_prompt, process_data

connections.connect(host='localhost', port='19530')
milvus_client = MilvusClient("http://localhost:19530")
collection_name = "my_rag_collection"

In [62]:
collection_name="my_rag_collection"
collection = Collection(collection_name)
collection.num_entities 

10662

In [17]:
utility.list_collections()

['question_answer', 'my_rag_collection']

In [18]:
res = collection.query(expr="", output_fields=["count(*)"])
print(res)

[{'count(*)': 10662}]


In [19]:
embedding_model = 'models/text-embedding-004'
genai.configure(api_key=API_KEY)
def embed_fn(title, text):
  return genai.embed_content(model=embedding_model,
                             content=text,
                             task_type="retrieval_document",
                             title=title)["embedding"]

In [50]:
question = "How Much Should I spend on Motorcycle Insurance ? "

search_res = milvus_client.search(
    collection_name=collection_name,
    data=[
        embed_fn(question,question)
    ],  # Use the `emb_text` function to convert the question to an embedding vector
    limit=7,  # Return top 3 results
    search_params={"metric_type": "IP", "params": {}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)

In [51]:
retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
retrieved_lines_with_distances

[(['How Much Does Motorcycle Insurance Cost? ',
   "Just like car insurance , there are many factors that are used to develop a premium for Motorcycle Insurance . Your age , driving record , credit history and location as well as the year make and model of the bike are all such factors . In addition , if you have other business with the insurance company like your homeowners and car insurance , multi policy discounts might apply . I am in Connecticut and have all my insurance with the same company and pay $ 220.00 for the year for full coverage and high liability limits on my '08 Suzuki Boulevard C50 . "],
  0.8232659697532654),
 (['Does Motorcycle Insurance Cost More Than Car Insurance? ',
   'This would be the most general answer possible but over all motorcycle is less then auto . But there are many many factors that can cause rates to go either way as it pertains to usage and vehicle itself . Simple example is on the motorcycle policy you are insuring a 2012 25k in value Harley and

In [52]:
querry_data= process_data(retrieved_lines_with_distances)
print(querry_data)

How Much Does Motorcycle Insurance Cost?
Just like car insurance , there are many factors that are used to develop a premium for Motorcycle Insurance . Your age , driving record , credit history and location as well as the year make and model of the bike are all such factors . In addition , if you have other business with the insurance company like your homeowners and car insurance , multi policy discounts might apply . I am in Connecticut and have all my insurance with the same company and pay $ 220.00 for the year for full coverage and high liability limits on my '08 Suzuki Boulevard C50 .

Does Motorcycle Insurance Cost More Than Car Insurance?
This would be the most general answer possible but over all motorcycle is less then auto . But there are many many factors that can cause rates to go either way as it pertains to usage and vehicle itself . Simple example is on the motorcycle policy you are insuring a 2012 25k in value Harley and the auto policy a 2000 Toyota Corolla . Driving

In [53]:
testing_question=""
prompt=create_prompt(querry_data,question)
print(prompt)


Task: Answer insurance-related questions accurately and provide some short explanation
Instructions:
Read the document carefully.
Consider key points, concepts, and details within the document.
Generate questions that seek information or clarification about the document's content.
Ensure that the questions are grammatically correct and understandable.
If the answer is not provided in the context, return None.
Context:
How Much Does Motorcycle Insurance Cost?
Just like car insurance , there are many factors that are used to develop a premium for Motorcycle Insurance . Your age , driving record , credit history and location as well as the year make and model of the bike are all such factors . In addition , if you have other business with the insurance company like your homeowners and car insurance , multi policy discounts might apply . I am in Connecticut and have all my insurance with the same company and pay $ 220.00 for the year for full coverage and high liability limits on my '08 S

In [54]:
LLM_model = genai.GenerativeModel('gemini-pro')
response = LLM_model.generate_content(prompt)

In [55]:
print( response.text.replace("JSON", "").replace("```",""))

The cost of motorcycle insurance varies depending on factors such as age, driving record, credit history, location, and the year, make, and model of the motorcycle. It is recommended to shop around with different insurance agents to compare rates and coverage options.
